In [1]:
# ignore sys warning
import warnings
import os
import glob
warnings.filterwarnings('ignore')

In [2]:
### visualization
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.offline as py
import matplotlib as mat
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.models import Range1d, LinearAxis, NumeralTickFormatter, ColumnDataSource, ranges, LabelSet, Legend, SingleIntervalTicker, LinearAxis
from bokeh.layouts import row
from bokeh.charts import Bar
from bokeh.palettes import brewer
py.init_notebook_mode(connected=True)

C:\Users\jiawei\Anaconda3\lib\site-packages\bokeh\util\deprecation.py:34: BokehDeprecationWarning:


The bokeh.charts API has moved to a separate 'bkcharts' package.

This compatibility shim will remain until Bokeh 1.0 is released.
After that, if you want to use this API you will have to install
the bkcharts package explicitly.




In [3]:
# other library
import datetime
import calendar

In [4]:
# data analysis and wrangling
import numpy as np
import pandas as pd
#import re
from functools import partial

# LTV

電商經營最重要的一環在於客戶價值的最大化，因此我們需要一個工具去幫我們檢視客戶的貢獻價值，而Life Time Value(LTV)便是一個適合的工具。

## Load order data from csv files

In [5]:
path =r'C:\Users\jiawei\Desktop\odr_line_data' 
allFiles = glob.glob(path + "/*.csv")
df = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df_temp = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df_temp)
    del df_temp
df = pd.concat(list_)

選出想要分析的商城，並去除取消的訂單

In [6]:
df = df.loc[df.order_status != 'Cancel']
df_sel = df.loc[df.shop_id == 1993].reset_index().drop('index', axis=1)
del df

### Select useful columns

選出需要使用的欄位

In [7]:
df_sel = df_sel[['cart_code', 'member_code', 'order_time', 'order_label', 'payment', 
                 'cost', 'qty', 'promotion_discount', 'ecoupon_discount']]
df_sel['product_cost'] = df_sel['cost'] * df_sel['qty']
df_sel.drop(['cost', 'qty'], axis=1, inplace=True)

In [8]:
df_sel.head(3)

,cart_code,member_code,order_time,order_label,payment,promotion_discount,ecoupon_discount,product_cost
0,TG160229231193,37509,2016-02-29T15:53:06.153000Z,Repo,390,0,0,270.0
1,TG160229231193,37509,2016-02-29T15:53:06.153000Z,Repo,350,0,0,0.0
2,TG160229231193,37509,2016-02-29T15:53:06.153000Z,Repo,350,0,0,198.0


## time format

將日期格是由字串轉為以日為單位的時間格式

In [9]:
time_format = '%Y-%m-%d'
df_sel['order_time'] = df_sel.order_time.astype(str).str[0:10]
df_sel['order_time'] = pd.to_datetime(df_sel['order_time'], format=time_format)
df_sel.head(3)

,cart_code,member_code,order_time,order_label,payment,promotion_discount,ecoupon_discount,product_cost
0,TG160229231193,37509,2016-02-29,Repo,390,0,0,270.0
1,TG160229231193,37509,2016-02-29,Repo,350,0,0,0.0
2,TG160229231193,37509,2016-02-29,Repo,350,0,0,198.0


## TS TO TG

由於訂單資訊包含商品資訊，因此會出現單筆訂單多個商品(列)的情況，由於LTV不需要那麼細緻的資料，因此我們會把銅筆訂單的資訊彙整起來。

In [10]:
df_tg = df_sel.groupby(['cart_code', 'member_code', 'order_time', 'order_label'], as_index=False).sum()
del df_sel
df_tg.tail(3)

,cart_code,member_code,order_time,order_label,payment,promotion_discount,ecoupon_discount,product_cost
466458,TG171031Z02361,196828,2017-10-31,Repo,628,-168,-100,485.0
466459,TG171031Z02368,15088,2017-10-31,Repo,710,0,-70,546.0
466460,TG171031Z02417,267313,2017-10-31,Repo,724,-254,0,549.0


## Calculate buy times

這邊我們計算每位消費者的購買次數

In [11]:
#print(df_tg.shape)
df_tg['times'] = df_tg.groupby('member_code', as_index=False).cumcount()+1
#print(df_tg.shape)
df_tg.tail(3)

,cart_code,member_code,order_time,order_label,payment,promotion_discount,ecoupon_discount,product_cost,times
466458,TG171031Z02361,196828,2017-10-31,Repo,628,-168,-100,485.0,2
466459,TG171031Z02368,15088,2017-10-31,Repo,710,0,-70,546.0,11
466460,TG171031Z02417,267313,2017-10-31,Repo,724,-254,0,549.0,3


## Get member first buy date time

這邊我們取得消費者第一次購買的日期

In [12]:
#print(df_tg.shape)
df_tg = pd.merge(df_tg,
                 df_tg.groupby('member_code', as_index=False)['order_time'].min(),
                 'left',
                 'member_code')
df_tg.columns = ['cart_code', 'member_code', 'order_time', 'order_label', 'payment',
                 'promotion_discount', 'ecoupon_discount', 'product_cost', 'times',
                 'first_buy_date_time']
#print(df_tg.shape)
df_tg.head(3)

,cart_code,member_code,order_time,order_label,payment,promotion_discount,ecoupon_discount,product_cost,times,first_buy_date_time
0,TG151209140050,1,2015-12-09,New,500,0,0,0.0,1,2015-12-09
1,TG151212140443,1,2015-12-12,Repo,445,0,0,0.0,2,2015-12-09
2,TG151212140639,1,2015-12-12,Repo,250,0,0,0.0,3,2015-12-09


## Get max_day & basemonth

這邊取得每位會員計算貢獻的期間(這邊以一年呈現)，以及依照每位會員的首購日的月份作為基準月份

In [13]:
def add_months(months, rows):
    month = rows['first_buy_date_time'].month - 1 + months
    year = int(rows['first_buy_date_time'].year + month / 12 )
    month = month % 12 + 1
    day = min(rows['first_buy_date_time'].day, calendar.monthrange(year, month)[1])
    return datetime.date(year, month, day)

In [14]:
# maxmonth
df_tg['maxmonth_1y'] = df_tg.apply(partial(add_months, 12), axis=1)
df_tg['maxmonth_1y'] = pd.to_datetime(df_tg['maxmonth_1y'], format=time_format)
df_tg.head(3)

,cart_code,member_code,order_time,order_label,payment,promotion_discount,ecoupon_discount,product_cost,times,first_buy_date_time,maxmonth_1y
0,TG151209140050,1,2015-12-09,New,500,0,0,0.0,1,2015-12-09,2016-12-09
1,TG151212140443,1,2015-12-12,Repo,445,0,0,0.0,2,2015-12-09,2016-12-09
2,TG151212140639,1,2015-12-12,Repo,250,0,0,0.0,3,2015-12-09,2016-12-09


In [15]:
# get basemonth
df_tg['basemonth'] = df_tg['first_buy_date_time'].astype(str).str[0:7]
#df_tg['basemonth'] = pd.to_datetime(df_tg['basemonth'], format=time_format)
df_tg['month'] = df_tg['order_time'].astype(str).str[0:7]
df_tg['month'] = pd.to_datetime(df_tg['month'], format=time_format)
df_tg.head(3)

,cart_code,member_code,order_time,order_label,payment,promotion_discount,ecoupon_discount,product_cost,times,first_buy_date_time,maxmonth_1y,basemonth,month
0,TG151209140050,1,2015-12-09,New,500,0,0,0.0,1,2015-12-09,2016-12-09,2015-12,2015-12-01
1,TG151212140443,1,2015-12-12,Repo,445,0,0,0.0,2,2015-12-09,2016-12-09,2015-12,2015-12-01
2,TG151212140639,1,2015-12-12,Repo,250,0,0,0.0,3,2015-12-09,2016-12-09,2015-12,2015-12-01


## Calculate total buy times

這邊我們要統計每位顧客至今為止總交易次數

In [16]:
df_tg = pd.merge(left=df_tg,
                 right=df_tg.groupby('member_code', as_index=False)['times'].max(),
                 how='left',
                 on='member_code')
df_tg.columns = ['cart_code', 'member_code', 'order_time', 'order_label', 'payment',
                 'promotion_discount', 'ecoupon_discount', 'product_cost', 'times',
                 'first_buy_date_time', 'maxmonth_1y', 'basemonth', 'month', 'max_times']

In [17]:
df_tg.head(3)

,cart_code,member_code,order_time,order_label,payment,promotion_discount,ecoupon_discount,product_cost,times,first_buy_date_time,maxmonth_1y,basemonth,month,max_times
0,TG151209140050,1,2015-12-09,New,500,0,0,0.0,1,2015-12-09,2016-12-09,2015-12,2015-12-01,21
1,TG151212140443,1,2015-12-12,Repo,445,0,0,0.0,2,2015-12-09,2016-12-09,2015-12,2015-12-01,21
2,TG151212140639,1,2015-12-12,Repo,250,0,0,0.0,3,2015-12-09,2016-12-09,2015-12,2015-12-01,21


## Filter order_time <= maxmonth_1y

在計算LTV之前，首先要將超過計算周期的交易紀錄去除，這樣才能讓每個月的比較有一樣的基準，週期大多數會訂在一年，但實際狀況依商城或服務而定。

In [18]:
df_tg_select = df_tg.loc[df_tg['order_time'] <= df_tg['maxmonth_1y']]
#print(df_tg_select.shape)
df_tg_select.sample(3)

,cart_code,member_code,order_time,order_label,payment,promotion_discount,ecoupon_discount,product_cost,times,first_buy_date_time,maxmonth_1y,basemonth,month,max_times
34095,TG160421230967,212259,2016-04-21,Repo,925,-45,0,587.0,2,2016-03-07,2017-03-07,2016-03,2016-04-01,3
292018,TG170203Z00458,194012,2017-02-03,New,563,-58,-25,438.0,1,2017-02-03,2018-02-03,2017-02,2017-02-01,1
230855,TG161127Z01580,42709,2016-11-27,New,730,0,-50,525.0,1,2016-11-27,2017-11-27,2016-11,2016-11-01,2


## Calculate each usr payment & cost

這邊主要計算每個月份的每一位會員的貢獻

In [19]:
df_same_piv = pd.pivot_table(df_tg_select, 
                             index=['member_code', 'basemonth'], 
                             values=['payment', 'product_cost', 'promotion_discount', 'ecoupon_discount'], 
                             aggfunc=np.sum).reset_index()
#print(df_same_piv.shape)
df_same_piv.head(5)

,member_code,basemonth,ecoupon_discount,payment,product_cost,promotion_discount
0,1,2015-12,-150,7903,2450.0,-72
1,2,2015-12,-50,6216,1537.0,-525
2,3,2015-12,-432,2962,1591.0,0
3,4,2015-12,0,3234,1009.0,0
4,5,2015-12,-100,508,0.0,0


## Calculate month LTV

計算每個月份進來的顧客未來一年的貢獻金額與成本

In [20]:
df_month_piv = pd.pivot_table(df_same_piv, index=['basemonth'], 
                              values=['payment', 'product_cost', 'ecoupon_discount', 'promotion_discount'], 
                              aggfunc=np.mean).reset_index()

In [21]:
df_month_piv.head(3)

,basemonth,ecoupon_discount,payment,product_cost,promotion_discount
0,2015-12,-113.570225,2152.126404,800.238202,-90.609551
1,2016-01,-89.417227,1754.414536,647.104172,-67.165545
2,2016-02,-37.895706,1786.015337,765.339264,-43.917178


## Remove uncompleted month

這邊我們要先將一些未累積一年的月份與先前平台的其間資料給去除

In [22]:
df_month_piv = df_month_piv.iloc[3:(len(df_month_piv.index)-12),]
df_month_piv.tail(3)

,basemonth,ecoupon_discount,payment,product_cost,promotion_discount
8,2016-08,-18.744102,1318.433004,753.547257,-63.079082
9,2016-09,-26.766452,1358.839365,810.968257,-99.818106
10,2016-10,-28.956346,1280.437887,776.745947,-95.098581


## Calculate Net LTV

這邊我們先扣除已知的訂單相關成本來得到Net LTV，知道這個數字之後，我們可以根據我們所需的獲利結構來規劃後續的行銷費用佔比

In [23]:
df_month_piv.payment = np.round(df_month_piv.payment, 2)
df_month_piv.product_cost = np.round(df_month_piv.product_cost, 2)
df_month_piv.ecoupon_discount = np.round(df_month_piv.ecoupon_discount, 2)
df_month_piv.promotion_discount = np.round(df_month_piv.promotion_discount, 1)
df_month_piv.ecoupon_discount = -df_month_piv.ecoupon_discount
df_month_piv.promotion_discount = -df_month_piv.promotion_discount
df_month_piv['net_LTV'] = df_month_piv.payment - df_month_piv.product_cost - df_month_piv.ecoupon_discount - \
                          df_month_piv.promotion_discount
df_month_piv['LTV'] = df_month_piv['payment']
df_month_piv['total_discount'] = df_month_piv.ecoupon_discount + df_month_piv.promotion_discount
df_month_piv.drop(['payment', 'ecoupon_discount', 'promotion_discount'], axis=1, inplace=True)

In [24]:
df_month_piv

,basemonth,product_cost,net_LTV,LTV,total_discount
3,2016-03,855.56,563.83,1527.03,107.64
4,2016-04,814.50,530.47,1446.49,101.52
5,2016-05,735.22,503.21,1305.67,67.24
6,2016-06,818.78,567.40,1465.52,79.34
7,2016-07,758.35,525.22,1369.71,86.14
8,2016-08,753.55,483.04,1318.43,81.84
9,2016-09,810.97,421.30,1358.84,126.57
10,2016-10,776.75,379.63,1280.44,124.06


## Draw LTV 

In [25]:
# bar-plot basic
# 調整座標軸
output_notebook()

# source for pic
source1 = ColumnDataSource(dict(x=df_month_piv['basemonth'].tolist(),y=df_month_piv['LTV'].tolist()))
source2 = ColumnDataSource(dict(x=df_month_piv['basemonth'].tolist(),y=np.round(df_month_piv['net_LTV'].tolist(), 2)))
source3 = ColumnDataSource(dict(x=df_month_piv['basemonth'].tolist(),y=np.round(df_month_piv['product_cost'].tolist(), 2)))
source4 = ColumnDataSource(dict(x=df_month_piv['basemonth'].tolist(),y=np.round(df_month_piv['total_discount'].tolist(), 2)))

# define pic size
p = figure(plot_width=1500, plot_height=800, 
           #x_axis_type=datetime, 
           title='Each Month LTV vs CPA', 
           #x_axis_label='Month', 
           #y_axis_label='LTV',
           x_range = source1.data["x"],
           y_range= ranges.Range1d(start=0,end=1600))

# LTV
p.vbar(x=df_month_piv['basemonth'].tolist(), 
       top=df_month_piv.LTV,
       width=0.5, 
       bottom=0, 
       color='blue', 
       legend='LTV')

# net_LTV
p.line(df_month_piv['basemonth'].tolist(), 
       df_month_piv['net_LTV'], 
       legend = 'net_LTV', 
       color="red",
       line_width=2)

p.circle(df_month_piv['basemonth'].tolist(), 
         df_month_piv['net_LTV'], 
         legend = 'net_LTV', 
         fill_color="red", 
         size=8)

# product_cost
p.line(df_month_piv['basemonth'].tolist(), 
       df_month_piv['product_cost'], 
       legend = 'product_cost', 
       color="black",
       line_width=2)

p.circle(df_month_piv['basemonth'].tolist(),
         df_month_piv['product_cost'], 
         legend = 'product_cost',
         color="black",
         size=8)


# total_discount
p.line(df_month_piv['basemonth'].tolist(), 
       df_month_piv['total_discount'], 
       legend = 'total_discount', 
       color="orange",
       line_width=2)

p.circle(df_month_piv['basemonth'].tolist(), 
         df_month_piv['total_discount'], 
         legend = 'total_discount', 
         color="orange",
         size=8)

# labels
labels1 = LabelSet(x='x', 
                  y='y', 
                  text='y', 
                  level='glyph', 
                  x_offset=-30, 
                  y_offset=0, 
                  text_font_size='17pt',
                  text_font_style='bold',
                  text_color='blue',
                  source=source1, 
                  render_mode='canvas')
p.add_layout(labels1)


#p.add_layout(LinearAxis(y_range_name="bar2"), 'right')
p.yaxis[0].formatter = NumeralTickFormatter(format="$0,0")
#p.yaxis[1].formatter = NumeralTickFormatter(format="$0")
p.legend.location = 'bottom_left'
p.background_fill_color = "white"
p.title.text_font_size = '12pt'
p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'
show(p, notebook_handle=True)

Loading BokehJS ...

![LTV_1Y EXAMPLE](LTV_1Y.png)